In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

## ODE Solution

In [ ]:
def forward_euler(ddt, u0, T, *args):
    u = np.empty((T.size, u0.size))
    u[0] = u0
    for i in range(1, T.size):
        u[i] = u[i-1] + (T[i] - T[i-1]) * ddt(u[i-1], T[i-1], *args)
    return u

def ddt(u, t, params):
    beta, rho, sigma = params
    x, y, z = u
    return np.array([sigma*(y-x), x*(rho-z)-y, x*y-beta*z])

def solve_ode(N, dt, u0, params=[8/3, 28, 10]):
    """
        Solves the ODEs for N time steps starting from u0.
        Returned values are normalized.

        Args:
            N: number of time steps
            u0: initial condition
            norm: normalisation factor of u0 (None if not normalised)
            params: parameters for ODE
        Returns:
            normalized time series of shape (N+1, u0.size)
    """

    T = np.arange(N+1) * dt
    U = forward_euler(ddt, u0, T, params)

    return U

## ESN 

In [ ]:
## ESN with bias architecture

def step(x_pre, u, sigma_in, rho):
    """ Advances one ESN time step.
        Args:
            x_pre: reservoir state
            u: input
        Returns:
            new augmented state (new state with bias_out appended)
    """
    # input is normalized and input bias added
    u_augmented = np.hstack((u/norm, np.array([bias_in]))) 
    # hyperparameters are explicit here
    x_post      = np.tanh(np.dot(u_augmented*sigma_in, Win) + rho*np.dot(x_pre, W)) 
    # output bias added
    x_augmented = np.hstack((x_post, np.array([bias_out])))

    return x_augmented

# @njit(parallel=False)
def open_loop(U, x0, sigma_in, rho):
    """ Advances ESN in open-loop.
        Args:
            U: input time series
            x0: initial reservoir state
        Returns:
            time series of augmented reservoir states
    """
    N  = U.shape[0]
    Xa = np.empty((N+1, N_units+1))
    Xa[0] = np.hstack((x0, np.array([bias_out])))
    for i in np.arange(1,N+1):
        Xa[i] = step(Xa[i-1,:N_units], U[i-1], sigma_in, rho)

    return Xa

# @njit(parallel=False)
def closed_loop(N, x0, Wout, sigma_in, rho):
    """ Advances ESN in closed-loop.
        Args:
            N: number of time steps
            x0: initial reservoir state
            Wout: output matrix
        Returns:
            time series of prediction
            final augmented reservoir state
    """
    xa = x0.copy()
    Yh = np.empty((N+1, dim))
    Yh[0] = np.dot(xa, Wout)
    for i in np.arange(1,N+1):
        xa = step(xa[:N_units], Yh[i-1], sigma_in, rho)
        Yh[i] = np.dot(xa, Wout)

    return Yh, xa

def train(U_washout, U_train, Y_train, tikh, sigma_in, rho):
    """ Trains ESN.
        Args:
            U_washout: washout input time series
            U_train: training input time series
            tikh: Tikhonov factor
        Returns:
            time series of augmented reservoir states
            optimal output matrix
    """
    
    LHS = 0
    RHS = 0
    
    N  = U_train[0].shape[0]    
    Xa  = np.zeros((U_washout.shape[0], N+1, N_units+1))
    
    for i in range(U_washout.shape[0]):
        
        ## washout phase
        xf_washout = open_loop(U_washout[i], np.zeros(N_units), sigma_in, rho)[-1,:N_units]

        ## open-loop train phase
        Xa[i] = open_loop(U_train[i], xf_washout, sigma_in, rho)
    
        ## Ridge Regression
        LHS  += np.dot(Xa[i,1:].T, Xa[i,1:])
        RHS  += np.dot(Xa[i,1:].T, Y_train[i])
    
    Wout = np.linalg.solve(LHS + tikh*np.eye(N_units+1), RHS)

    return Xa[0], Wout, LHS, RHS

In [2]:
def predictability_horizon(xa, Y, Wout):
    """ Compute predictability horizon. It evolves the network until the
        error is greater than the threshold. Before that it initialises
        the network by running a washout phase.
        
        Args:
            threshold: error threshold
            U_washout: time series for washout
            Y: time series to compare prediction
        
        Returns:
            predictability horizon (in time units, not Lyapunov times)
            time series of normalised error
            time series of prediction
    """
    
    # calculate denominator of the normalised error
    kin   = 0.5*np.linalg.norm(Y, axis=1)**2
#     err_z = np.sqrt(np.mean(kin**2))
    err_d = np.mean(np.sum((Y[:,:dim])**2, axis=1))
    
    N     = Y.shape[0]
    E     = np.zeros(N+1)
    E_z   = np.zeros(N+1)
    Yh    = np.zeros((N, len(idx1)))
    kinh  = np.zeros(N)
    
    Yh[0]   = np.dot(xa, Wout)
    kinh[0] =0.5*np.linalg.norm(Yh[0])**2
    
    pr_idx = np.zeros(N)
    pr_idd = np.zeros(N)
    
    pr_idx[-1] = 1
    pr_idd[-1] = 1
    
    for i in range(1, N):
        # advance one step
        xa       = step(xa[:N_units], Yh[i-1][:dim])
        Yh[i]    = np.dot(xa, Wout)
        kinh[i]  = 0.5*np.linalg.norm(Yh[i])**2
    
        # calculate error
        E_z[i]  = np.abs(kinh[i]-kin[i])
        err_n   = np.sum(((Yh[i,:dim]-Y[i,:dim]))**2)
        E[i]    = np.sqrt(err_n/err_d)

        if E_z[i] > threshold:
            break
        
        if E_z[i] > tt:
            pr_idx[i] = 1
            
        if E[i]   > 0.2:
            pr_idd[i] = 1
    
    a = np.nonzero(pr_idx)[0][0]/N_lyap
    b = np.nonzero(pr_idd)[0][0]/N_lyap

    t = np.arange(i)/N_lyap
#     fig, ax1 = plt.subplots(1,2)
    
#     ax=plt.subplot(1,2,1)
#     plt.annotate('{:.2f}'.format(a), xy=(0, 1), xytext=(5, -5), va='top', ha='left',
#              xycoords='axes fraction', textcoords='offset points')
#     plt.ylim(1e-4,threshold)
#     plt.axhline(.2)
#     ax.set_ylabel('$E$')
#     plt.plot(t,E_z[:i], label='$E_k$')
#     plt.plot(t,E[:i], label='$E$')
#     plt.legend()
    
#     ax = plt.subplot(1,2,2)
#     ax.set_ylabel('$k$')
    if is_plot:
        plt.axhline(ee,linewidth=3, alpha=0.3)
        plt.axvline(a,linewidth=3, alpha=0.3)
        plt.plot(t,kin[:i], label='True')
        plt.plot(t,kinh[:i], label='ESN')
        plt.legend(fontsize=20)
        plt.show()
            
    return a, b

In [ ]:
def predictability_horizon_k(xa, Y, Wout, sigma_in, rho):
    """ Compute predictability horizon. It evolves the network until the
        error is greater than the threshold. Before that it initialises
        the network by running a washout phase.
        
        Args:
            threshold: error threshold
            U_washout: time series for washout
            Y: time series to compare prediction
        
        Returns:
            predictability horizon (in time units, not Lyapunov times)
            time series of normalised error
            time series of prediction
    """
    
    # calculate denominator of the normalised error
    kin   = 0.5*np.linalg.norm(Y, axis=1)**2
    
    N     = Y.shape[0]
    E     = np.zeros(N+1)
    Yh    = np.zeros((N, len(idx1)))
    kinh  = np.zeros(N)
    
    Yh[0]   = np.dot(xa, Wout)
    kinh[0] = 0.5*np.linalg.norm(Yh[0])**2
    
    pr_idx = np.zeros(N)
    pr_idd = np.zeros(N)
    
    pr_idx[-1] = 1
    pr_idd[-1] = 1
    
    for i in range(1, N):
        # advance one step
        xa       = step(xa[:N_units], Yh[i-1][:dim], sigma_in, rho)
        Yh[i]    = np.dot(xa, Wout)
        kinh[i]  = 0.5*np.linalg.norm(Yh[i])**2
    
        # calculate error
        E[i]  = np.abs(kinh[i]-kin[i])
            
        if E[i] > threshold:
            break
#             pr_idd[i] = 1
    
#     a = np.nonzero(pr_idx)[0][0]/N_lyap
#     b = np.nonzero(pr_idd)[0][0]/N_lyap

#     t = np.arange(i)/N_lyap
#     fig, ax1 = plt.subplots(1,2)
    
#     ax=plt.subplot(1,2,1)
#     plt.annotate('{:.2f}'.format(a), xy=(0, 1), xytext=(5, -5), va='top', ha='left',
#              xycoords='axes fraction', textcoords='offset points')
#     plt.ylim(1e-4,threshold)
#     plt.axhline(.2)
#     ax.set_ylabel('$E$')
#     plt.plot(t,E_z[:i], label='$E_k$')
#     plt.plot(t,E[:i], label='$E$')
#     plt.legend()
    
#     ax = plt.subplot(1,2,2)
#     ax.set_ylabel('$k$')
#     if is_plot:
#         plt.axhline(ee,linewidth=3, alpha=0.3)
#         plt.axvline(a,linewidth=3, alpha=0.3)
#         plt.plot(t,kin[:i], label='True')
#         plt.plot(t,kinh[:i], label='ESN')
#         plt.legend(fontsize=20)
#         plt.show()
            
    return i/N_lyap

In [ ]:
def MFE_modes(X):
    
    Ndim = 9
    # Problem Definition
    Lx = 4*math.pi
    Ly = 2.
    Lz = 2*math.pi
    Re = 400
    # Parameter values
    alfa  = 2*math.pi/Lx
    beta  = math.pi/2
    gamma = 2*math.pi/Lz

    pi       = math.pi
    cos      = np.cos
    sin      = np.sin

    x, y, z  = X

    k1 = np.sqrt(alfa**2 + gamma**2)
    k2 = np.sqrt(gamma**2 + beta**2)
    k3 = np.sqrt(alfa**2 + beta**2 + gamma**2) 
    
    # a1, a2, a3, a4, a5, a6, a7, a8, a9 = q.T 
    
    m1  = np.array([np.sqrt(2)*sin(pi*y/2), 0*x, 0*x])

    m2  = np.array([4/np.sqrt(3)*cos(pi*y/2)**2*cos(gamma*z), 0*x, 0*x]) 

    m3  = 2/np.sqrt(4*gamma**2+pi**2)*np.array(
         [0*x, 2*gamma*cos(pi*y/2)*cos(gamma*z), pi*sin(pi*y/2)*sin(gamma*z)]) 

    m4  = np.array([0*x, 0*x, 4/np.sqrt(3)*cos(alfa*x)*cos(pi*y/2)**2]) 

    m5  = np.array([0*x, 0*x, 2*sin(alfa*x)*sin(pi*y/2)])

    m6  = 4*np.sqrt(2)/np.sqrt(3*(gamma**2+alfa**2))*np.array(
        [-gamma*cos(math.pi*y/2)**2*cos(alfa*x)*sin(gamma*z), 0*x,
          alfa*cos(math.pi*y/2)**2*sin(alfa*x)*cos(gamma*z)]) 

    m7  = 2*np.sqrt(2)/np.sqrt(gamma**2+alfa**2)*np.array(
        [ gamma*sin(math.pi*y/2)*sin(alfa*x)*sin(gamma*z), 0*x,
          alfa*sin(math.pi*y/2)*cos(alfa*x)*cos(gamma*z)])

    N8  = 2*np.sqrt(2)/np.sqrt((gamma**2+alfa**2)*(4*alfa**2+4*gamma**2+pi**2))

    m8  = N8*np.array([pi*alfa*sin(math.pi*y/2)*sin(alfa*x)*sin(gamma*z), 
                       2*(gamma**2+alfa**2)*cos(math.pi*y/2)*cos(alfa*x)*sin(gamma*z),
                       -pi*gamma*sin(math.pi*y/2)*cos(alfa*x)*cos(gamma*z)]) 

    m9  = np.array([np.sqrt(2)*sin(3*math.pi*y/2), 0*x, 0*x])
    
    return np.stack([m1,m2,m3,m4,m5,m6,m7,m8,m9],axis=0)